# Import libraries

In [ ]:
!pip install rdkit

In [ ]:
import rdkit, rdkit.Chem, rdkit.Chem.Draw
from rdkit.Chem import Descriptors
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import pairwise_distances_argmin_min
import matplotlib

# Read the database

In [ ]:
!wget https://raw.githubusercontent.com/stefano-bosio/CTF_ML_MD/main/kinase-examples.smi
!wget https://raw.githubusercontent.com/stefano-bosio/CTF_ML_MD/main/lgic-examples.smi

In [ ]:
suppl_kinases=rdkit.Chem.rdmolfiles.SmilesMolSupplier('kinase-examples.smi')
kinases=[x for x in suppl_kinases]

In [ ]:
suppl_lgic=rdkit.Chem.rdmolfiles.SmilesMolSupplier('lgic-examples.smi')
lgic=[x for x in suppl_lgic]

In [ ]:
features_kinases=pd.DataFrame()
for i,mol in enumerate(kinases):
    features_kinases.loc[i,'MolWt']=Descriptors.MolWt(mol)
    features_kinases.loc[i,'NumHAcceptors']=Descriptors.NumHAcceptors(mol)
    features_kinases.loc[i,'NumHDonors']=Descriptors.NumHDonors(mol)
    features_kinases.loc[i,'NumRotBonds']=Descriptors.NumRotatableBonds(mol)
    features_kinases.loc[i,'NumHeteroatoms']=Descriptors.NumHeteroatoms(mol)
    features_kinases.loc[i,'FractionCSP3']=Descriptors.FractionCSP3(mol)
    features_kinases.loc[i,'RingCount']=Descriptors.RingCount(mol)
    features_kinases.loc[i,'TPSA']=Descriptors.TPSA(mol)
    features_kinases.loc[i,'Stereocenters']=rdkit.Chem.rdMolDescriptors.CalcNumAtomStereoCenters(mol)
    features_kinases.loc[i,'Spiro']=rdkit.Chem.rdMolDescriptors.CalcNumSpiroAtoms(mol)
    features_kinases.loc[i,'NumArR']=Descriptors.NumAromaticRings(mol)
    features_kinases.loc[i,'NumAliR']=Descriptors.NumAliphaticRings(mol)
    features_kinases.loc[i,'set']=0

In [ ]:
features_kinases

In [ ]:
features_lgic=pd.DataFrame()
for i,mol in enumerate(lgic):
    features_lgic.loc[i,'MolWt']=Descriptors.MolWt(mol)
    features_lgic.loc[i,'NumHAcceptors']=Descriptors.NumHAcceptors(mol)
    features_lgic.loc[i,'NumHDonors']=Descriptors.NumHDonors(mol)
    features_lgic.loc[i,'NumRotBonds']=Descriptors.NumRotatableBonds(mol)
    features_lgic.loc[i,'NumHeteroatoms']=Descriptors.NumHeteroatoms(mol)
    features_lgic.loc[i,'FractionCSP3']=Descriptors.FractionCSP3(mol)
    features_lgic.loc[i,'RingCount']=Descriptors.RingCount(mol)
    features_lgic.loc[i,'TPSA']=Descriptors.TPSA(mol)
    features_lgic.loc[i,'Stereocenters']=rdkit.Chem.rdMolDescriptors.CalcNumAtomStereoCenters(mol)
    features_lgic.loc[i,'Spiro']=rdkit.Chem.rdMolDescriptors.CalcNumSpiroAtoms(mol)
    features_lgic.loc[i,'NumArR']=Descriptors.NumAromaticRings(mol)
    features_lgic.loc[i,'NumAliR']=Descriptors.NumAliphaticRings(mol)
    features_lgic.loc[i,'set']=1

In [ ]:
features_lgic

In [ ]:
features=pd.concat((features_kinases,features_lgic))

In [ ]:
features

# Feature Selection

### Scaling of variables

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
features_scaled=pd.DataFrame(MinMaxScaler().fit_transform(features), columns=features.columns)

## 1. Univariate Linear Filtering

In [ ]:
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

In [ ]:
array = features_scaled.values

In [ ]:
X = array[:,0:12]
Y = array[:,12]
feat=features.columns

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
test = SelectKBest(score_func=mutual_info_classif)
fit = test.fit(X, Y)

In [ ]:
scores=pd.DataFrame(feat[:-1],columns=['feat'])
scores['scores']=fit.scores_
scores.sort_values(by='scores', ascending=False, inplace=True)
print(scores)

## 2. Wrapping through Recursive Feature Elimination

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(solver='lbfgs', max_iter=5000)
data=[]

for i in range(len(feat[:-1])):
  n_feat=len(feat[:-1])-i

  rfe = RFE(model, n_features_to_select=n_feat)   # seek for n_feat features
  fit = rfe.fit(X, Y)

  data.append((fit.ranking_))
  scores=pd.DataFrame(feat[:-1],columns=['feat'])
  scores['scores']=fit.ranking_
  scores.sort_values(by='scores',ascending=True,inplace=True)

  print(f"Model with {n_feat} features")
  print(scores)

In [ ]:
model = LogisticRegression(solver='lbfgs', max_iter=5000)

rfe = RFECV(model)   # seek for best number of features
fit = rfe.fit(X, Y)

scores=pd.DataFrame(feat[:-1],columns=['feat'])
scores['scores']=fit.ranking_
scores.sort_values(by='scores',ascending=True,inplace=True)

print(f"Model with {fit.n_features_} features")
print(scores)

In [ ]:
plt.plot(range(1,len(feat)),fit.cv_results_['mean_test_score'],marker='o')
plt.xlabel("# Features")
plt.ylabel("CV score")

# Feature Extraction aka Dimensionality Reduction

## Principal Component Analysis

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca=PCA()

In [ ]:
low_dim=pca.fit_transform(X)

In [ ]:
low_dim.shape

In [ ]:
low_dim[:,0].shape

In [ ]:
plt.bar(np.linspace(0,len(pca.explained_variance_ratio_), num=len(pca.explained_variance_ratio_)),pca.explained_variance_ratio_)
plt.xlabel("Eigenvector #")
plt.ylabel("Explained Variance Ratio")

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(low_dim[:1999,0], low_dim[:1999,1])
plt.scatter(low_dim[2000:,0], low_dim[2000:,1])
plt.xlabel("PC1")
plt.ylabel("PC2")

In [ ]:
from sklearn.cluster import *

In [ ]:
kmeans=KMeans(n_clusters=8,random_state=8).fit(low_dim[:,:2])

In [ ]:
np.unique(kmeans.labels_)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(low_dim[:,0], low_dim[:,1],c=kmeans.labels_)
plt.xlabel("PC1")
plt.ylabel("PC2")

In [ ]:
tot=kinases+lgic

In [ ]:
len(tot)

In [ ]:
closest_points= pairwise_distances_argmin_min(kmeans.cluster_centers_, low_dim[:,:2])
closest_points

In [ ]:
low_dim[closest_points[0]].shape

In [ ]:
low_dim[closest_points[0],0]

In [ ]:
low_dim[closest_points[0],1]

In [ ]:
it=0
plt.figure(figsize=(8,8))
plt.scatter(low_dim[:,0], low_dim[:,1],c=kmeans.labels_)
for i in closest_points[0]:
  it+=1
  plt.scatter(low_dim[i,0], low_dim[i,1],marker='o',s=100,edgecolor="black",color="red" )
  plt.annotate(it, (low_dim[i,0]+0.05, low_dim[i,1]+0.05), color="black", fontsize=15)

plt.xlabel("PC1")
plt.ylabel("PC2")

In [ ]:
subset=[]
for i in closest_points[0]:
  subset.append(tot[i])

In [ ]:
subset[0]

In [ ]:
img=rdkit.Chem.Draw.MolsToGridImage(subset,molsPerRow=4,subImgSize=(300,300), legends=[str(x) for x in np.arange(1,9,1)])
img

## Non-linear dimensionality reduction methods

### 1. ISOMAP

In [ ]:
from sklearn.manifold import Isomap

In [ ]:
iso=Isomap(n_neighbors=12)

In [ ]:
data_red=iso.fit_transform(X)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(data_red[:1999,0], data_red[:1999,1])
plt.scatter(data_red[2000:,0], data_red[2000:,1])
plt.xlabel("ISOMAP1")
plt.ylabel("ISOMAP2")

In [ ]:
kmeans=KMeans(n_clusters=8,random_state=8).fit(data_red[:,:2])

In [ ]:
np.unique(kmeans.labels_)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(data_red[:,0], data_red[:,1],c=kmeans.labels_)
plt.xlabel("ISOMAP1")
plt.ylabel("ISOMAP2")

In [ ]:
closest_points= pairwise_distances_argmin_min(kmeans.cluster_centers_, data_red[:,:2])
closest_points

In [ ]:
it=0
plt.figure(figsize=(8,8))
plt.scatter(data_red[:,0], data_red[:,1],c=kmeans.labels_)
for i in closest_points[0]:
  it+=1
  plt.scatter(data_red[i,0], data_red[i,1],marker='o',s=100,edgecolor="black",color="red" )
  plt.annotate(it, (data_red[i,0]+0.1, data_red[i,1]+0.1), color="black", fontsize=15)

plt.xlabel("ISOMAP1")
plt.ylabel("ISOMAP2")

In [ ]:
subset=[]
for i in closest_points[0]:
  subset.append(tot[i])

In [ ]:
subset[0]

In [ ]:
img=rdkit.Chem.Draw.MolsToGridImage(subset,molsPerRow=4,subImgSize=(300,300), legends=[str(x) for x in np.arange(1,9,1)])
img

### 2. t-SNE

In [ ]:
from sklearn.manifold import  *

In [ ]:
tsne=TSNE(perplexity=np.sqrt(len(X)))

In [ ]:
data_red=tsne.fit_transform(X)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(data_red[:1999,0], data_red[:1999,1])
plt.scatter(data_red[2000:,0], data_red[2000:,1])
plt.xlabel("t-SNE1")
plt.ylabel("t-SNE2")

In [ ]:
kmeans=KMeans(n_clusters=8,random_state=8).fit(data_red[:,:2])

In [ ]:
np.unique(kmeans.labels_)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(data_red[:,0], data_red[:,1],c=kmeans.labels_)
plt.xlabel("t-SNE1")
plt.ylabel("t-SNE2")

In [ ]:
closest_points= pairwise_distances_argmin_min(kmeans.cluster_centers_, data_red[:,:2])
closest_points

In [ ]:
it=0
plt.figure(figsize=(8,8))
plt.scatter(data_red[:,0], data_red[:,1],c=kmeans.labels_)
for i in closest_points[0]:
  it+=1
  plt.scatter(data_red[i,0], data_red[i,1],marker='o',s=100,edgecolor="black",color="red" )
  plt.annotate(it, (data_red[i,0]+0.5, data_red[i,1]+0.5), color="black", fontsize=15)

plt.xlabel("t-SNE1")
plt.ylabel("t-SNE2")

In [ ]:
subset=[]
for i in closest_points[0]:
  subset.append(tot[i])

In [ ]:
subset[0]

In [ ]:
img=rdkit.Chem.Draw.MolsToGridImage(subset,molsPerRow=4,subImgSize=(300,300), legends=[str(x) for x in np.arange(1,9,1)])
img

In [ ]:
distance=np.sqrt(np.square(data_red[:,0]-(data_red[closest_points[0][2],0]))+np.square(data_red[:,1]-(data_red[closest_points[0][2],1])))

In [ ]:
nearest_points=np.argsort(distance,axis=0)[:50]

In [ ]:
nearest_points

In [ ]:
from rdkit import Chem

In [ ]:
Chem.MolToSmiles(tot[0])

In [ ]:
nearest_mols=[]
unique_smiles=[]
for i in nearest_points:
  if Chem.MolToSmiles(tot[i]) not in unique_smiles:
    nearest_mols.append(tot[i])
    unique_smiles.append(Chem.MolToSmiles(tot[i]))


In [ ]:
img=rdkit.Chem.Draw.MolsToGridImage(nearest_mols,molsPerRow=3,subImgSize=(300,300))
img

# Classification

## 1. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import DecisionBoundaryDisplay

In [ ]:
x_reduced=data_red[:,0]

In [ ]:
x_reduced

In [ ]:
Y

In [ ]:
clr=LogisticRegression()

In [ ]:
clr.fit(x_reduced.reshape(-1,1),Y)

In [ ]:
clr.coef_

In [ ]:
clr.intercept_

In [ ]:
plt.scatter(x_reduced,Y,alpha=0.05,c=Y,cmap=matplotlib.colors.ListedColormap(["C0", "C1"]))
plt.xlabel("t-SNE1")
plt.ylabel("Class")

In [ ]:
x=np.linspace([x_reduced.min()-1,x_reduced.max()+1],100)

In [ ]:
response=np.exp(clr.intercept_+clr.coef_*x)/(np.exp(clr.intercept_+clr.coef_*x)+1)

In [ ]:
plt.scatter(x_reduced,Y,alpha=0.05,c=Y,cmap=matplotlib.colors.ListedColormap(["C0", "C1"]))
plt.plot(x,response,'--',c='k',lw=2)
plt.xlabel("t-SNE1")
plt.ylabel("Class")

## 2. Linear Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
lda=LinearDiscriminantAnalysis()

In [ ]:
x_reduced=data_red[:,:2]

In [ ]:
lda.fit(x_reduced,Y)

In [ ]:
feature_1, feature_2 = np.meshgrid(
    np.linspace(x_reduced[:, 0].min(), x_reduced[:, 0].max()),
    np.linspace(x_reduced[:, 1].min(), x_reduced[:, 1].max()))

In [ ]:
grid = np.vstack([feature_1.ravel(), feature_2.ravel()]).T

In [ ]:
y_predict=np.reshape(lda.predict(grid),feature_1.shape)

In [ ]:
display = DecisionBoundaryDisplay(xx0=feature_1, xx1=feature_2, response=y_predict)

In [ ]:
display.plot(cmap=matplotlib.colors.ListedColormap(["C0", "C1"]))
display.ax_.scatter(x_reduced[:, 0], x_reduced[:, 1], c=Y, edgecolor="k",cmap=matplotlib.colors.ListedColormap(["C0", "C1"]))
plt.xlabel("t-SNE1")
plt.ylabel("t-SNE2")

## 3. Support Vector Machines

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
svc=SVC(kernel='linear')
#svc=SVC(kernel='poly', degree=2)
#svc=SVC(kernel='poly', degree=3)
#svc=SVC(kernel='poly', degree=4)
#svc=SVC(kernel='sigmoid')
#svc=SVC(kernel='rbf')

In [ ]:
svc.fit(x_reduced,Y)

In [ ]:
y_predict=np.reshape(svc.predict(grid),feature_1.shape)

In [ ]:
display = DecisionBoundaryDisplay(xx0=feature_1, xx1=feature_2, response=y_predict)

In [ ]:
display.plot(cmap=matplotlib.colors.ListedColormap(["C0", "C1"]))
display.ax_.scatter(x_reduced[:, 0], x_reduced[:, 1], c=Y, edgecolor="k",cmap=matplotlib.colors.ListedColormap(["C0", "C1"]))
plt.xlabel("t-SNE1")
plt.ylabel("t-SNE2")